The Trace Inequality
===

Let $\Omega \subset {\mathbb R}^d$ with a Lipschitz-continuous boundary. If $u \in H^1(\Omega)$, then $u$ has well-defined boundary
values $u_{|\partial \Omega}$, called the trace of $u$. There holds

$$
\| u_{|\partial \Omega} \|_{L_2(\partial \Omega)} \leq c \, \| u \|_{H^1(\Omega)}
$$

We call $\operatorname{tr} : u \mapsto u_{|\partial \Omega}$ the trace operator.

Numerical verification
---
We setup two bilinear-forms

\begin{eqnarray*}
T(u,v) & = & \int_{\partial \Omega} u v \, ds \\
N(u,v) & = & \int_{\Omega} \nabla u \nabla v + u v \; dx
\end{eqnarray*}

for which there holds

$$
\| \operatorname{tr} \|_{H^1(\Omega)\rightarrow L_2(\partial\Omega)}^2 = \sup_{u \in H^1} \frac{T(u,u)}{N(u,u)}.
$$

Which is the largest eigenvalue of the generalized eigenvalue-problem

$$
T u = \lambda N u,
$$

or the smallest eigenvalue of 

$$
N u = \frac{1}{\lambda} T u.
$$

We use the PINVIT - algorithm in NGSolve to compute the smallest few eigenvalue of the last evp:

In [ ]:
from ngsolve import *
from ngsolve.meshes import MakeStructured2DMesh
from ngsolve.webgui import Draw 

mesh = MakeStructured2DMesh(nx=10,ny=10, mapping = lambda x,y : (x,y))
fes = H1(mesh, order=3)
u,v = fes.TnT()

H1Norm = BilinearForm(grad(u)*grad(v)*dx+u*v*dx).Assemble()
TraceNorm = BilinearForm(u*v*ds).Assemble()
pre = H1Norm.mat.Inverse()
evals,evecs = solvers.PINVIT(H1Norm.mat, TraceNorm.mat, pre=pre, num=5, maxit=10, printrates=False)
print ("eigenvalues: ", list(evals))
print ("Norm of trace operator: ||tr||^2 = ", 1/evals[0])

gfu = GridFunction(fes)
gfu.vec.data = evecs[0]
gfu.vec.data /= Integrate(gfu*dx, mesh) # normalize eigenfunction
Draw (gfu, mesh)

Do experiments with
* the discretization parameters
* scaling of the $\nabla u \nabla v$ and $u v$ terms in the $H^1$-norm
* the domain size 
* if we scale the domain-size by a factor $a$, then we have to scale the norms as ... to obtain the same $\| \operatorname{tr} \|$.
* investigate the first and second eigenvalues

Proof of the trace inequality:
---
The construction of the trace operator follows the following plan:
* We use that $C^1$ is dense in $H^1$. 
* We prove the inequality in $C^1$.
* For $u \in H^1$ we choose a sequence $(u_n)$ in $C^1$ such that
$u_n \rightarrow u$ in $H^1$.
* This $(u_n)$ is a Cauchy-sequence in $H^1$-norm. Since the trace inequality is proven in $C^1$, the sequence $(u_{n|\partial \Omega})$ is Cauchy in $L_2(\partial \Omega)$.
* Finally we define $u_{|\partial \Omega}$ as the limit of $(u_{n|\partial \Omega})$. It does not depend on the chosen approximation sequence.


Proof in 1D:
---
Let $\Omega = (0,a)$. 